# Set up

In [19]:
# # import spotipy
# # from spotipy import util
# # from spotipy.oauth2 import SpotifyClientCredentials
# from get_spotify_client import get_spotify_obj
# from utility_functions import create_connection
# from utility_functions  import execute_sql_query
# import data_pull_functions as pull

`client_id`: client ID from your project app on the spotify's app [dashboard](https://developer.spotify.com/dashboard)

`client_secret`: client secret from your project app on the spotify's app [dashboard](https://developer.spotify.com/dashboard)

`client_credentials_manager`: This create's a client to access the spotify API.  Since no access token was created, only public data may be accessed and no user interaction is available.

`spotipy.Spotify`: This is an object to send requests to the spotify API

## Getting Spotify Client

In [20]:
# sp = get_spotify_obj()

# Get the [top 50 global playist](https://open.spotify.com/playlist/37i9dQZEVXbMDoHDwVN2tF?si=87c0069b7da449b7)

`playlist_tracks` - Grabs a spotify playlist as `dict`.  In this case, we grab the _global top 50_.  Please call the `items` kvp to get a list of the tracks contained in the playlist

In [21]:
# # pulls a playlist object & returns the attribute 'items'. 'items' is the list of track objects for tracks in playlist
# top_fifty_global_tracks = sp.playlist_tracks("https://open.spotify.com/playlist/37i9dQZEVXbMDoHDwVN2tF?si=87c0069b7da449b7",
#                                       fields='items')


# top_fifty_global_tracks = top_fifty_global_tracks.get('items')



Current fields we need for playlist relation:

* `track_name` - The name of the track lol
* `track_id` - The track's id
* `album_name` - The name of the album the track is a part of
* `album_id` - The album's id
* `date_added` - `DateTime` this was loaded this tuple was loaded to our database
 

The following is an example of a track in our playlist.  Each track returns as a dict and we want to highlight the various information we oculd get.

In [22]:
# track_one = top_fifty_global_tracks[1]
# track_one.get('track').get('artists')
# track_one

The track's `track` kvp is where we get the information of the `track_id`, `track`, `album` columns

In [23]:
# print("Track Name: " + track_one.get('track').get('name'))
# print("Track Id: " + track_one.get('track').get('id'))

the track's `album` kvp is where we can get the track's `album` and `album_id`

In [24]:
# print("Album Name: " + track_one.get('track').get('album').get('name'))
# print("Album Id: " + track_one.get('track').get('album').get('id'))

# Grabbing top 50 tracks in playlist

`get_global_top_fifty_data` returns a `list` of `tuple` where each tuple is a track data in the format `(track, track_id, album, album_id)`


In [25]:
# def get_global_top_fifty_song_album_data(sp):
#     playlist_id = "https://open.spotify.com/playlist/37i9dQZEVXbMDoHDwVN2tF?si=87c0069b7da449b7"
#     top_fifty_global = sp.playlist_tracks(playlist_id)

# # This is a list of all tracks in the playlist.  Track objects are dictionaries
#     top_fifty_global_tracks = top_fifty_global.get("items")

#     tuples = []
    
#     for song in top_fifty_global_tracks:
#         added_it = song.get('added_at')
        
#         song_info = song.get('track')
#         track = song_info.get('name')
#         track_id = song_info.get('id')
    
#         album_info = song_info.get('album')
#         album = album_info.get('name')
#         album_id = album_info.get('id')

#         date_loaded = str(date.today())
    
#         row = (track, track_id, album, album_id, date_loaded)
#         tuples.append(row)

#     return tuples

# Getting track data for track table

`get_all_song_ids` returns a list of all the unique tracks and their ids for the track table as a `dict`.

In [26]:
# def get_all_song_ids(list_of_tracks):
#     track_dict = {}
#     ids = set()
    
#     for track in list_of_tracks:
#         track_id = track[1]
#         track_name = track[0]
        
#         if track_id not in ids:
#             track_dict.update({track_id:track_name})
#         else:
#             continue
#     return track_dict

In [27]:
#data = get_global_top_fifty_song_album_data(sp)

# Grabbing the artist data

`get_all_top_fifty_artists` returns the artist that related to all of the songs in a playlist

In [28]:
# def get_all_top_fifty_artists(sp):
#     playlist_id = "https://open.spotify.com/playlist/37i9dQZEVXbMDoHDwVN2tF?si=87c0069b7da449b7"
#     top_fifty_global = sp.playlist_tracks(playlist_id)
#     top_fifty_global_tracks = top_fifty_global.get("items")
#     tuples = []

#     for track in top_fifty_global_tracks:
#         # This is a list of dicts containing where each dict is an artist's information
#         track_artists = track.get('track').get('artists')
#         track_artists_data = [(artist.get('id'), artist.get('name')) for artist in track_artists]
#         tuples += track_artists_data

#     unique_artists = set(tuples)
#     tuples = list(tuples)
#     return tuples

# Grabbing the album data

`get_all_top_fifty_albums` returns the artist that related to all of the songs in a playlist

In [29]:
# def get_all_top_fifty_albums(sp):
#     playlist_id = "https://open.spotify.com/playlist/37i9dQZEVXbMDoHDwVN2tF?si=87c0069b7da449b7"
#     top_fifty_global = sp.playlist_tracks(playlist_id)
#     top_fifty_global_tracks = top_fifty_global.get("items")
#     tuples = []

#     for track in top_fifty_global_tracks:
#         # This is a list of dicts containing where each dict is an artist's information
#         album_id = track.get('track').get('album').get('id')
#         album_name = track.get('track').get('album').get('name')
#         row =(album_id, album_name)
#         tuples.append(row)
        
#     return tuples

# Inserting data

## Insert Tracks

In [30]:
# def insert_tracks(data, conn):
#     get_current_data_query = """
#     SELECT * FROM Tracks;
#     """
    
#     insert_query = """
#     INSERT INTO Tracks (track_id, track_name) VALUES (?, ?)
#     """
    
#     cur = conn.cursor()
#     with conn:
        
#     # grab current data
#         current_data = execute_sql_query(get_current_data_query, conn)
#         current_data_set = set(current_data)
#         data_set = set(data)
#         new_data = data_set - current_data_set
#         new_rows = list(new_data)
#     # to insert data
#         cur.executemany(insert_query, new_rows)
#         conn.commit()
        

In [31]:
# def insert_albums(data, conn):
#     get_current_data_query = """
#     SELECT * FROM Albums;
#     """
    
#     insert_query = """
#     INSERT INTO Albums (album_id, album_name) VALUES (?, ?)
#     """
    
#     cur = conn.cursor()
#     with conn:
        
#     # grab current data
#         current_data = execute_sql_query(get_current_data_query, conn)
#         current_data_set = set(current_data)
#         data_set = set(data)
#         new_data = data_set - current_data_set
#         new_rows = list(new_data)
#     # to insert data
#         cur.executemany(insert_query, new_rows)
#         conn.commit()

In [32]:
# def insert_artists(data, conn):
#     get_current_data_query = """
#     SELECT * FROM Artists;
#     """
    
#     insert_query = """
#     INSERT INTO Artists (artist_id, artist_name) VALUES (?, ?)
#     """
    
#     cur = conn.cursor()
#     with conn:
        
#     # grab current data
#         current_data = execute_sql_query(get_current_data_query, conn)
#         current_data_set = set(current_data)
#         data_set = set(data)
#         new_data = data_set - current_data_set
#         new_rows = list(new_data)
#     # to insert data
#         cur.executemany(insert_query, new_rows)
#         conn.commit()

In [33]:
# def insert_playlist(data, conn):
#     get_current_data_query = """
#     SELECT * FROM Playlist;
#     """
    
#     insert_query = """
#     INSERT INTO Playlist (track_id, track_name, album_id, album_name, date_loaded) VALUES (?, ?, ?, ?, ?)
#     """
    
#     cur = conn.cursor()
#     with conn:
        
#     # grab current data
#         current_data = execute_sql_query(get_current_data_query, conn)
#         current_data_set = set(current_data)
#         data_set = set(data)
#         new_data = data_set - current_data_set
#         new_rows = list(new_data)
#     # to insert data
#         cur.executemany(insert_query, new_rows)
#         conn.commit()

In [34]:
# conn = create_connection('SpotifyData.db')
# playlist = pull.get_global_top_fifty_playlist(sp)
# insert_playlist(data=playlist, conn=conn)


# New Code
Each function below will grab data for a particular table or are helper functions to grab data for a particular table

## Setup

In [35]:
# import spotipy
# from spotipy import util
# from spotipy.oauth2 import SpotifyClientCredentials
from get_spotify_client import get_spotify_obj
from utility_functions import create_connection
from utility_functions  import execute_sql_query
import data_pull_functions as pull

sp = get_spotify_obj()

## Raw Data Pull

`get_playlist_data` gets the data raw data for a playlist.   The raw data includes the name of the playlists and all information related to tracks, artists, and albums with the exception of audio analysis, market code definitions, key signature encodings, and other audio analysis metrics.

In [36]:
def get_playlist_data(sp, link):
    """
    inputs:
        sp: A spotify object
        link: url to the spotify playlist
    outputs:
        dictionary of two keys (name, followers, items)
        'name' is the name of the playlist
        'items' is a list of attributes to upload in playlists 
        Track objects are one of those attributes
        Track objects are just dict
    """

    playlist = sp.playlist(link)
    playlist_name = playlist.get('name')
    playlist_items = playlist.get('tracks').get('items')
    playlist_data = {'name': playlist_name, 'items': playlist_items}
    return playlist_data

`get_audio_features` gets the audio features of a track or list of tracks

In [37]:
def get_audio_features(sp, tracks: list[str]):
    """
    inputs:
        sp: A spotify object
        tracks: A list of spotify track URIs
          to return audio features for (maximum number of tracks to pull data in one call is 100)
    outputs:
        a dictionary with track IDs as keys and audio features as values
    """
    if len(tracks) > 50:
        raise Exception("Too many tracks in 'tracks' argument")
    else:
        print('I GOT HERE')
        data = sp.audio_features(tracks)
        updated_data = {}
        for feature in data:
            track_id = feature.pop("id")
            updated_data.update({track_id:feature})
        return updated_data

## Get Table Data

In [38]:
test_data = get_playlist_data(sp, 'https://open.spotify.com/playlist/37i9dQZEVXbLRQDuF5jeBp?si=778db61081ae460f')

`get_playlist_table_data` is a function to get the data for `Playlist`

In [39]:
def get_playlist_table_data(playlist_data):
    """
    inputs:
        playlist_data: Output of get_playlist_data()
            This is the raw data of a playlist
    outputs:
        table_data:  These are the rows of for Playlist
    """
# Name of playlist
    name = playlist_data.get("name")
# List of items
    items = playlist_data.get("items")
    
# Table Data
    table_data = []
    for item in items:
    # Get the track obj
        track = item.get('track')
        
        track_id = track.get('id')
        track_name = track.get('name')
        album_id = track.get('album').get('id')
        album_name = track.get('album').get('name')
        row = (name, track_id, track_name, album_id, album_name)
        table_data.append(row)
    return table_data

`get_tracks_table_data` gets the data for `Tracks`

In [40]:
def get_tracks_table_data(sp, playlist_data):
    """
    inputs:
        sp: A spotify object
        playlist_data: Output of get_playlist_data()
            This is the raw data of a playlist
    outputs:
        table_data:  These are the rows of for Tracks
    """
# List of items
    items = playlist_data.get("items")
    
# Table Data
    table_data = []

    for item in items:
    # Get the track obj
        track = item.get('track')

    # Get track info from track
        track_id = track.get('id')
        track_name = track.get('name')
        album_id = track.get('album').get('id')
        duration_ms = track.get('duration_ms')
        explicit = track.get('explicit')
        current_popularity = track.get('popularity')
        row = (track_id, track_name, album_id, duration_ms, explicit, current_popularity)
        table_data.append(row)
    
# Use track_id to pull audio features
    get_ids = lambda tup: tup[0]
    ids = [get_ids(tup) for tup in table_data]
    audio_features = get_audio_features(sp, ids)

# update each row in table_data
    table_data_length = len(table_data)
    for index in range(table_data_length):
        row = table_data[index]
        track_id = row[0]
        track_audio_features = audio_features.get(track_id)
        acousticness = track_audio_features.get('acousticness')
        danceability = track_audio_features.get('danceability')
        instrumentalness = track_audio_features.get('instrumentalness')
        key_signature = track_audio_features.get('key')
        mode = track_audio_features.get('mode')
        tempo = track_audio_features.get('tempo')
        time_signature = track_audio_features.get('time_signature')
        valence = track_audio_features.get('valence')
        audio_feature_attributes = (acousticness, danceability, instrumentalness,
                                    key_signature, mode, tempo, time_signature,
                                    valence)
        updated_row = row + audio_feature_attributes
        table_data[index] = updated_row
    return table_data

`get_albums_table_data` gets the data for `Albums`

In [41]:
def get_Albums_table_data(playlist_data):
    """
    inputs:
        playlist_data: Output of get_playlist_data()
            This is the raw data of a playlist
    outputs:
        table_data:  These are the rows of for Albums
    """

# List of items
    items = playlist_data.get("items")
    
# Table Data
# table_data is a dict to ensure return of unique album IDs since multiple tracks from an album
# can be in a playlist
    table_data = []
    found_albums = set()
    for item in items:
    # Get the track obj
        track = item.get('track')
        album = track.get('album')
        album_id = album.get('id')
        album_name = album.get('name')
        total_tracks = album.get('total_tracks')
        date_released = album.get('release_date')
        row = (album_id, album_name, total_tracks, date_released)
        
        if album_id not in found_albums:
            table_data.append(row)
            found_albums.add(album_id)
        else:
            continue

    return table_data

`get_AlbumArtists_table_data` gets the data for `AlbumArtists`

In [42]:
def get_AlbumArtists_table_data(playlist_data):
    """
    inputs:
        playlist_data: Output of get_playlist_data()
            This is the raw data of a playlist
    outputs:
        table_data:  These are the rows of for AlbumArtists
    """

# List of items
    items = playlist_data.get("items")
    
# Table Data
    table_data = []
    for item in items:
    # Get the track obj
        track = item.get('track')
        album = track.get('album')
        album_id = album.get('id')
    
    # artists is a list of artist obj
        artists = album.get('artists')
        artists_ids = [artist.get('id') for artist in artists]
        for artist_id in artists_ids:
            row = (album_id, artist_id)
            table_data.append(row)

    table_data = set(table_data)
    table_data = list(table_data)
    return table_data

`get_TrackArtists_table_data` gets the data for `TrackArtists`

In [43]:
def get_TrackArtists_table_data(playlist_data):
    """
    inputs:
        playlist_data: Output of get_playlist_data()
            This is the raw data of a playlist
    outputs:
        table_data:  These are the rows of for TrackArtists
    """

# List of items
    items = playlist_data.get("items")
    
# Table Data
    table_data = []
    for item in items:
    # Get the track obj
        track = item.get('track')
        track_id = track.get('id')
        
    # artists is a list of artist obj
        artists = track.get('artists')
        artists_ids = [artist.get('id') for artist in artists]
        
        for artist_id in artists_ids:
            row = (track_id, artist_id)
            table_data.append(row)

    table_data = set(table_data)
    table_data = list(table_data)
    return table_data

`get_TrackAvailableMarkets_table_data` gets the data for `TrackAvailableMarkets`

In [44]:
def get_TrackAvailableMarkets_table_data(playlist_data):
    """
    inputs:
        playlist_data: Output of get_playlist_data()
            This is the raw data of a playlist
    outputs:
        table_data:  These are the rows of for TrackAvailableMarkets
    """

# List of items
    items = playlist_data.get("items")
    
# Table Data
    table_data = []
    for item in items:
    # Get the track obj
        track = item.get('track')
        track_id = track.get('id')
        available_markets = track.get('available_markets')
        
        for market in available_markets:
            row = (track_id, market)
            table_data.append(row)

    table_data = set(table_data)
    table_data = list(table_data)
    return table_data

`get_GenreArtists_table_data` gets the data for `GenreArtists`

In [45]:
def get_GenreArtists_table_data(sp, playlist_data):
    """
    inputs:
        sp: A spotify object
        playlist_data: Output of get_playlist_data()
        This is the raw data of a playlist
    outputs:
        table_data:  These are the rows of for GenreArtists
    """
# List of items
    items = playlist_data.get("items")
   
# Table Data
    table_data = []

# Grab all unique artist ids to get their genre later
    unique_artist_ids = []
    for item in items:
    # Get the track obj
        track = item.get('track')

    # artists is a list of artist obj
        artists = track.get('artists')
    # a dictionary of artist ids and their corresponding genres
        artist_ids = [artist.get('id') for artist in artists]
        unique_artist_ids += artist_ids

    unique_artist_ids = set(unique_artist_ids)
    unique_artist_ids = list(unique_artist_ids)

# artists returns a dictionary with one kvp "artists" containing each artist obj
    artist_genre_data = sp.artists(unique_artist_ids).get('artists')

    for artist_obj in artist_genre_data:
        artist_id = artist_obj.get('id')
        genres = artist_obj.get('genres')
    # create rows for each genre for an artist id
        rows = [(artist_id, genre) for genre in genres]
        table_data += rows
    
    table_data = set(table_data)
    table_data = list(table_data)
    return table_data

`get_Artists_table_data` gets the data for `Artists`

In [46]:
def get_Artists_table_data(sp, playlist_data):
    """
    inputs:
        sp: A spotify object
        playlist_data: Output of get_playlist_data()
            This is the raw data of a playlist
    outputs:
        table_data:  These are the rows of for GenreArtists
    """
# List of items
    items = playlist_data.get("items")
   
# Table Data
    table_data = []

# Grab all unique artist ids to get their genre later
    unique_artist_ids = []
    for item in items:
    # Get the track obj
        track = item.get('track')

    # artists is a list of artist obj
        artists = track.get('artists')
    # a dictionary of artist ids and their corresponding genres
        artist_ids = [artist.get('id') for artist in artists]
        unique_artist_ids += artist_ids

    unique_artist_ids = set(unique_artist_ids)
    unique_artist_ids = list(unique_artist_ids)

# artists returns a dictionary with one kvp "artists" containing each artist obj
    artist_genre_data = sp.artists(unique_artist_ids).get('artists')

    for artist_obj in artist_genre_data:
        artist_id = artist_obj.get('id')
        artist_name = artist_obj.get('name')
        nbr_of_followers = artist_obj.get('followers').get('total')
        popularity = artist_obj.get('popularity')
    # create rows for each genre for an artist id
        row = (artist_id, artist_name, nbr_of_followers, popularity)
        table_data.append(row)
    
    table_data = set(table_data)
    table_data = list(table_data)
    return table_data

In [49]:
import json
bad_line = 1
did_i_pull_all_playlists = []
with open('countries_top_fifty.txt', "r") as file:
        for line in file:
            spotify_id = 'spotify'
            row_split = line.strip("").split(",")
            link = row_split[1]
            playlist = sp.playlist(link)
            did_i_pull_all_playlists.append(playlist)
            print(playlist.get('name'))
            bad_line += 1
            if row_split[0] == 'USA':
                break

with open("playlist_sample.json", "w") as outfile: 
    json.dump(did_i_pull_all_playlists[2].get('tracks').get('items')[0].get('track'), outfile)

Top 50 - Japan
Top 50 - Israel
Top 50 - USA


In [50]:
def list_partition(some_list, size):
    partition = []
    if len(some_list) <= size:
        partition.append(some_list)
        return(partition)
    else:
        partition_one = [some_list[ele] for ele in range(size)]
        partition.append(partition_one)
        partition_two = [some_list[ele] for ele in range(size, len(some_list))]
        return partition + list_partition(partition_two, size)


In [51]:

test_ids = ['0AjmK0Eai4zGrLaJwPvrDp', '2GxrNKugF82CnoRFbQfzPf', '0mflMxspEfB0VbI1kyLiAv', '3xkHsmpQCBMytMJNiDf3Ii', '3qhlB30KknSejmIvZZLjOD', '17phhZDn6oGtzMe56NuWvj', '7wLShogStyDeZvL0a6daN5', '51ZQ1vr10ffzbwIjDCwqm4', '2uqYupMHANxnwgeiXTZXzd', '3Pbp7cUCx4d3OAkZSCoNvn', '3rUGC1vUpkDG9CZFHMur1t', '59NraMJsLaMCVtwXTSia8i', '2tudvzsrR56uom6smgOcSf', '5uQ7de4EWjb3rkcFxyEOpu', '3U5JVgI2x4rDyHGObzJfNf', '3w0w2T288dec0mgeZZqoNN', '1BxfuPKGuaTgP7aM0Bbdwr', '5Jpz4nv8nA6FIYO5gEyQa2', '6dpLxbF7lfCAnC9QRTjNLK', '6Qb7YsAqH4wWFUMbGsCpap', '4xhsWYTOGcal8zt0J161CU', '1bjeWoagtHmUKputLVyDxQ', '4OMJGnvZfDvsePyCwRGO7X', '19kHhX6f6EfLU7rcO3RqjO', '7aQ1tSYr518KwnE4SVSQbT', '7ueuqvQkKA9k1Pys0hB5br', '5aIVCx5tnk0ntmdiinnYvw', '6tNgRQ0K2NYZ0Rb9l9DzL8', '3dYD57lRAUcMHufyqn9GcI', '6me7F0aaZjwDo6RJ5MrfBD', '003vvx7Niy0yvhvHt4a68B', '1TcF5F10EN17vraisG1gQv', '51eSHglvG1RJXtL3qI5trr', '0LMwmV37RCmBO2so0szAFs', '4tKGFmENO69tZR9ahgZu48', '4KULAymBBJcPRpk1yO4dOG', '3hRV0jL3vUpRrcy398teAU', '1kuGVB7EU95pJObxwvfwKS', '5D34wRmbFS29AjtTOP2QJe', '1VxEaXbIq3xAvqh80xP67w', '1k1Bqnv2R0uJXQN4u6LKYt', '1ACA277B6f46DYCgZW8di3', '2Zo1PcszsT9WQ0ANntJbID', '7jtQIBanIiJOMS6RyCx6jZ', '42VsgItocQwOQC3XWZ8JNA', '50SxWFCvmZCYcpKWPZECES', '6zuPQl2RVqQ4y5nAH61CFC', '6kopmMZiyLmw7h66uXcXR7', '3vkCueOmm7xQDoJ17W1Pm3', '6wf7Yu7cxBSPrRlWeSeK0Q']
test_ = get_playlist_data(sp, 'https://open.spotify.com/playlist/37i9dQZEVXbKM896FDX8L1?si=84c205afc9934953')

In [ ]:
def get_tracks_table_data(sp, playlist_data):
    """
    inputs:
        sp: A spotify object
        playlist_data: Output of get_playlist_data()
            This is the raw data of a playlist
    outputs:
        table_data:  These are the rows of for Tracks
    """
# List of items
    items = playlist_data.get("items")
    
# Table Data
    table_data = []

    for item in items:
    # Get the track obj
        track = item.get('track')

    # Get track info from track
        track_id = track.get('id')
        track_name = track.get('name')
        album_id = track.get('album').get('id')
        duration_ms = track.get('duration_ms')
        explicit = track.get('explicit')
        current_popularity = track.get('popularity')
        row = (track_id, track_name, album_id, duration_ms, explicit, current_popularity)
        table_data.append(row)
    
# Use track_id to pull audio features
    get_ids = lambda tup: tup[0]
    ids = [get_ids(tup) for tup in table_data]
    print(len(ids))
    audio_features = get_audio_features(sp, ids)
# update each row in table_data
    table_data_length = len(table_data)
    for index in range(table_data_length):
        row = table_data[index]
        track_id = row[0]
        track_audio_features = audio_features.get(track_id)
        acousticness = track_audio_features.get('acousticness')
        danceability = track_audio_features.get('danceability')
        instrumentalness = track_audio_features.get('instrumentalness')
        key_signature = track_audio_features.get('key')
        mode = track_audio_features.get('mode')
        tempo = track_audio_features.get('tempo')
        time_signature = track_audio_features.get('time_signature')
        valence = track_audio_features.get('valence')
        audio_feature_attributes = (acousticness, danceability, instrumentalness,
                                    key_signature, mode, tempo, time_signature,
                                    valence)
        updated_row = row + audio_feature_attributes
        table_data[index] = updated_row
    return table_data



In [52]:
# get_tracks_table_data(sp, test_)
sp.tracks(['0AjmK0Eai4zGrLaJwPvrDp', '2GxrNKugF82CnoRFbQfzPf', '0mflMxspEfB0VbI1kyLiAv', '3xkHsmpQCBMytMJNiDf3Ii', '3qhlB30KknSejmIvZZLjOD', '17phhZDn6oGtzMe56NuWvj', '7wLShogStyDeZvL0a6daN5', '51ZQ1vr10ffzbwIjDCwqm4', '2uqYupMHANxnwgeiXTZXzd', '3Pbp7cUCx4d3OAkZSCoNvn', '3rUGC1vUpkDG9CZFHMur1t', '59NraMJsLaMCVtwXTSia8i', '2tudvzsrR56uom6smgOcSf', '5uQ7de4EWjb3rkcFxyEOpu', '3U5JVgI2x4rDyHGObzJfNf', '3w0w2T288dec0mgeZZqoNN', '1BxfuPKGuaTgP7aM0Bbdwr', '5Jpz4nv8nA6FIYO5gEyQa2', '6dpLxbF7lfCAnC9QRTjNLK', '6Qb7YsAqH4wWFUMbGsCpap', '4xhsWYTOGcal8zt0J161CU', '1bjeWoagtHmUKputLVyDxQ', '4OMJGnvZfDvsePyCwRGO7X', '19kHhX6f6EfLU7rcO3RqjO', '7aQ1tSYr518KwnE4SVSQbT', '7ueuqvQkKA9k1Pys0hB5br', '5aIVCx5tnk0ntmdiinnYvw', '6tNgRQ0K2NYZ0Rb9l9DzL8', '3dYD57lRAUcMHufyqn9GcI', '6me7F0aaZjwDo6RJ5MrfBD', '003vvx7Niy0yvhvHt4a68B', '1TcF5F10EN17vraisG1gQv', '51eSHglvG1RJXtL3qI5trr', '0LMwmV37RCmBO2so0szAFs', '4tKGFmENO69tZR9ahgZu48', '4KULAymBBJcPRpk1yO4dOG', '3hRV0jL3vUpRrcy398teAU', '1kuGVB7EU95pJObxwvfwKS', '5D34wRmbFS29AjtTOP2QJe', '1VxEaXbIq3xAvqh80xP67w', '1k1Bqnv2R0uJXQN4u6LKYt', '1ACA277B6f46DYCgZW8di3', '2Zo1PcszsT9WQ0ANntJbID', '7jtQIBanIiJOMS6RyCx6jZ', '42VsgItocQwOQC3XWZ8JNA', '50SxWFCvmZCYcpKWPZECES', '6zuPQl2RVqQ4y5nAH61CFC', '6kopmMZiyLmw7h66uXcXR7', '3vkCueOmm7xQDoJ17W1Pm3', '6wf7Yu7cxBSPrRlWeSeK0Q']).get('tracks')[0]

KeyboardInterrupt: 

In [53]:
sp.audio_features(['spotify:track:0AjmK0Eai4zGrLaJwPvrDp'])

[{'danceability': 0.741,
  'energy': 0.62,
  'key': 10,
  'loudness': -5.505,
  'mode': 1,
  'speechiness': 0.0412,
  'acousticness': 0.0295,
  'instrumentalness': 0.000809,
  'liveness': 0.0398,
  'valence': 0.934,
  'tempo': 117.038,
  'type': 'audio_features',
  'id': '0AjmK0Eai4zGrLaJwPvrDp',
  'uri': 'spotify:track:0AjmK0Eai4zGrLaJwPvrDp',
  'track_href': 'https://api.spotify.com/v1/tracks/0AjmK0Eai4zGrLaJwPvrDp',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0AjmK0Eai4zGrLaJwPvrDp',
  'duration_ms': 251424,
  'time_signature': 4}]

In [ ]:
def get_TrackAvailableMarkets_table_data(playlist_data):
    """
    Inputs:
        playlist_data: Output of get_playlist_data()
            This is the raw data of a playlist
    Outputs:
        table_data:  These are the rows of for TrackAvailableMarkets
    """

# List of items
    items = playlist_data.get("items")
    
# Table Data
    table_data = []
    for item in items:

    # Get the track obj
        track = item.get('track')
        track_id = track.get('id')
        
    # available_markets is a list of markets track is available in
        available_markets = track.get('available_markets')
        
        for market in available_markets:
            row = (track_id, market)
            table_data.append(row)

    return table_data

In [54]:
get_TrackAvailableMarkets_table_data(test_)

[('1VxEaXbIq3xAvqh80xP67w', 'BY'),
 ('3qhlB30KknSejmIvZZLjOD', 'BG'),
 ('6J2LdBN97cDWn0MLxYh9HB', 'MY'),
 ('59NraMJsLaMCVtwXTSia8i', 'ML'),
 ('4xhsWYTOGcal8zt0J161CU', 'MO'),
 ('6Qb7YsAqH4wWFUMbGsCpap', 'NG'),
 ('4tKGFmENO69tZR9ahgZu48', 'LV'),
 ('3xkHsmpQCBMytMJNiDf3Ii', 'GW'),
 ('6J2LdBN97cDWn0MLxYh9HB', 'TZ'),
 ('1VxEaXbIq3xAvqh80xP67w', 'PG'),
 ('3dYD57lRAUcMHufyqn9GcI', 'IS'),
 ('3w0w2T288dec0mgeZZqoNN', 'MT'),
 ('4tKGFmENO69tZR9ahgZu48', 'CM'),
 ('3rUGC1vUpkDG9CZFHMur1t', 'FJ'),
 ('4pcEACnm3ZfZNNepcxCT0c', 'LT'),
 ('0gEyKnHvgkrkBM6fbeHdwK', 'PL'),
 ('3xkHsmpQCBMytMJNiDf3Ii', 'LC'),
 ('59NraMJsLaMCVtwXTSia8i', 'BN'),
 ('1kuGVB7EU95pJObxwvfwKS', 'FM'),
 ('4tKGFmENO69tZR9ahgZu48', 'EG'),
 ('3Pbp7cUCx4d3OAkZSCoNvn', 'ID'),
 ('6dpLxbF7lfCAnC9QRTjNLK', 'DO'),
 ('3xkHsmpQCBMytMJNiDf3Ii', 'CV'),
 ('4sNG6zQBmtq7M8aeeKJRMQ', 'IS'),
 ('4KULAymBBJcPRpk1yO4dOG', 'SB'),
 ('3hRV0jL3vUpRrcy398teAU', 'RW'),
 ('6dpLxbF7lfCAnC9QRTjNLK', 'EE'),
 ('6dpLxbF7lfCAnC9QRTjNLK', 'BT'),
 ('0LMwmV37RCmBO2so0

In [55]:
from connect import connect
from config import load_config

In [56]:
config = load_config()
conn = connect(config)
cur = conn.cursor()
with conn:
    test_query = "DELETE FROM Tracks"
    cur.execute(test_query)

---- Successfully connected to PostgreSQL Server ----



In [57]:
def get_Artists_table_data(sp, playlist_data):
    """
    Inputs:
        sp: A spotify object
        playlist_data: Output of get_playlist_data()
            This is the raw data of a playlist
    Outputs:
        table_data:  These are the rows of for GenreArtists
    """
# List of items
    items = playlist_data.get("items")
   
# Table Data
    table_data = []

# Grab all unique artist ids to get their genre later
    unique_artist_ids = []
    for item in items:
    # Get the track obj
        track = item.get('track')

    # artists is a list of artist obj
        artists = track.get('artists')
    # a dictionary of artist ids and their corresponding genres
        artist_ids = [artist.get('id') for artist in artists]
        unique_artist_ids += artist_ids

# Make unique_artist_ids list
    unique_artist_ids = set(unique_artist_ids)
    unique_artist_ids = list(unique_artist_ids)

# Grab the total number of unique ids
    nbr_of_unique_ids = len(unique_artist_ids)

# Partition the data if we have more than 50 ids since url can only handle
# at most 50 ids
    if nbr_of_unique_ids > 50:

    # partition is a list of lists where each list contain ids
        partition = list_partition(unique_artist_ids, 50)
        for group in partition:
            artist_genre_data = sp.artists(group).get('artists')

            for artist_obj in artist_genre_data:
                artist_id = artist_obj.get('id')
                artist_name = artist_obj.get('name')
                nbr_of_followers = artist_obj.get('followers').get('total')
                popularity = artist_obj.get('popularity')

            # create rows for each genre for an artist id
                row = (artist_id, artist_name, nbr_of_followers, popularity)
                table_data.append(row)
        return table_data
    else:

    # artist_genre_data returns a dictionary with one kvp "artists" containing
    # each artist obj
        artist_genre_data = sp.artists(unique_artist_ids).get('artists')

        for artist_obj in artist_genre_data:
            artist_id = artist_obj.get('id')
            artist_name = artist_obj.get('name')
            nbr_of_followers = artist_obj.get('followers').get('total')
            popularity = artist_obj.get('popularity')

        # create rows for each genre for an artist id
            row = (artist_id, artist_name, nbr_of_followers, popularity)
            table_data.append(row)

        return table_data

In [58]:
get_Artists_table_data(sp, test_)

KeyboardInterrupt: 

In [59]:
sp.artist('0TnOYISbd1XYRBk9myaseg')

KeyboardInterrupt: 